#### Hello, and welcome  to the project. I am Abiodun Eesuola.

Let's start

To complete this assignment, I might follow this sequence:

| Task | To do | Tool |
|------|------|------|
| 1 | Transform `LGA_SEN_Districts.xls` → Structured Table | Python (pandas) or OpenRefine |
| 2 | Load & Join All Datasets | Python + GeoPandas |
| 3 | Link Health Facilities to Senatorial Districts | Spatial & Attribute Join |
| 4 | Find Best-Ranked Facility per Senatorial District | Pandas Aggregation |
| 5 | Export Final Data to SQLite | SQLAlchemy / sqlite3 |
| 6 | Create Map in QGIS | QGIS + Print Layout (A3 PDF) |
| 7 | Package Deliverables | Folder structure + Training Slide |


##### Objective:

- Transform by means of automation, the document ‘LGA_SEN_Districts’ into a well-structured database table that anyone can easily work with. 

- Export all final (processed) datasets in an SQLite database

- Make a map in QGIS showing the senatorial districts, highlighting health facilities with the best scores.

- Include the necessary map elements and export map in PDF and on A3 paper size.

**I will deliver:**

-> A well-structured table output from (1) above

-> The automation script or ETL tool used to generate (i) above

-> An SQLite database containing the spatial datasets I will be using for the map

-> A QGIS project showing the datasets and map

-> An A3 size exported PDF map of Nigeria senatorial districts showing health facilities ranking based on availability of personnel

-> A training slide for my team on how the process can be done.

Let's go!



#### Task 1
---
**At first glance, the file is stored as an .xls file. This is an old-school format. I will convert the file to an .xlsx file instead, while retaining all content. I achieved this using an online tool. The new file is saved as "LGA_SEN_Districts.xlsx" in the same folder.**

Next, I'll inspect the data to see it's structure and how to work with  it.

In [57]:
import pandas as pd

# Reading my file while automatically dropping empty rows and duplicates to perform initial cleaning.
df = pd.read_excel(r"c:\Users\USER\Desktop\eHealthOrg\Assignment 1\LGA_SEN_Districts.xlsx").dropna(how = "all").drop_duplicates()
display(df.head(50))


,NAME OF SENATORIAL DISTRICTS,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
1,ABIA,NaN,NaN,NaN,NaN
2,S/N,NAME OF SENATORIAL DISTRICT,CODE,COMPOSITION,NAME OF COLLATION CENTRE
3,1,ABIA NORTH,SD/001/AB,"UMUNNEOCHI, ISUKWUATO, OHAFIA, AROCHUKWU, BENDE",COUNCIL HALL OHAFIA LGA HQS
4,2,ABIA CENTRAL,SD/002/AB,"UMUAHIA NORTH, UMUAHIA SOUTH, IKWUANO, ISIALA ...",COUNCIL HALL UMUAHIA NORTH LGA HQS
5,3,ABIA SOUTH,SD/003/AB,"ABA NORTH, ABA SOUTH, UGWUNAGBO, OBINGWA, UKW...",ABA TOWN HALL ABA SOUTH LGA
7,AKWA IBOM,NaN,NaN,NaN,NaN
9,7,AKWA IBOM NORTH EAST,SD/007/AK,"ETINAN, IBESIKPO ASUTAN, IBIONO IBOM, ITU, NSI...",INEC OFFICE UYO LGA
10,8,AKWA IBOM NORTH WEST,SD/008/AK,"ABAK, ESSIEN UDIM, ETIM EKPO, IKA, IKONO, IKOT...",INEC OFFICE IKOT EKPENELGA
11,9,AKWA IBOM SOUTH,SD/009/AK,"EASTERN OBOLO, EKET, ESIT EKET, IBENO, IKOT AB...",EKET LGA INEC OFFICE
14,NaN,NaN,NaN,ADAMAWA,NaN



I see a pattern here.

1. Rows like **`ABIA`**, **`AKWA IBOM`**, **`ADAMAWA`**, etc. are **state names**, as seen right from the first row.

   * They appear in column `"NAME OF SENATORIAL DISTRICTS"`.
   * The other columns are empty on those rows.

2. The second row in the dataframe is a **header row** (`S/N, NAME OF SENATORIAL DISTRICT, CODE, COMPOSITION, NAME OF COLLATION CENTRE`).

3. After that come the **actual senatorial district rows** (`1, ABIA NORTH, SD/001/AB, ...`).


To clean this dataset into something usable, I need to:

- **Forward-fill state names** (so each senatorial district row inherits the correct state).
- **Ignore repeated header rows** (`S/N`, etc.).
- **Persistently drop empty rows** (like the NaN-only ones).
- **Rename the useful columns** to something like:

```python
["serial", "district", "code", "composition", "collation_centre", "state"]
```



In [64]:
import pandas as pd

# Load Excel (no header, so we can control cleaning)
df = pd.read_excel(
    r"c:\Users\USER\Desktop\eHealthOrg\Assignment 1\LGA_SEN_Districts.xlsx",
    header=None
)

# Rename temporary columns
df.columns = ["col1", "col2", "col3", "col4", "col5"]

# Detect rows that are state names (all caps words, not numeric, not "S/N")
state_rows = df["col1"].fillna("").str.strip()
df["state"] = df["col1"].where(
    state_rows.str.match(r"^[A-Z ]+$") & ~state_rows.str.contains("S/N", na=False)
)

# Forward-fill state names correctly
df["state"] = df["state"].ffill()

# Keep only rows where col1 is numeric → the districts
mask = df["col1"].fillna("").astype(str).str.isnumeric()
clean_df = df[mask].copy()

# Rename columns to meaningful names
clean_df = clean_df.rename(columns={
    "col2": "senatorial_district",
    "col3": "code",
    "col4": "composition",
    "col5": "collation_centre"
})

# Keep final useful columns
clean_df = clean_df[["state", "senatorial_district", "code", "composition", "collation_centre"]]

# I'm saving the cleaned files as both a csv and an Excel file for versatility, so anyone can easily work with it
clean_df.to_csv(r"c:\Users\USER\Desktop\eHealthOrg\Assignment 1\cleaned_LGA_SEN.csv", index=False)
clean_df.to_excel(r"c:\Users\USER\Desktop\eHealthOrg\Assignment 1\cleaned_LGA_SEN.xlsx", index=False)

# Summary: number of districts per state
summary = clean_df.groupby("state")["senatorial_district"].count().reset_index()
summary = summary.rename(columns={"senatorial_district": "num_districts"})

display(clean_df.head(), clean_df.tail())

,state,senatorial_district,code,composition,collation_centre
4,ABIA,ABIA NORTH,SD/001/AB,"UMUNNEOCHI, ISUKWUATO, OHAFIA, AROCHUKWU, BENDE",COUNCIL HALL OHAFIA LGA HQS
5,ABIA,ABIA CENTRAL,SD/002/AB,"UMUAHIA NORTH, UMUAHIA SOUTH, IKWUANO, ISIALA ...",COUNCIL HALL UMUAHIA NORTH LGA HQS
6,ABIA,ABIA SOUTH,SD/003/AB,"ABA NORTH, ABA SOUTH, UGWUNAGBO, OBINGWA, UKW...",ABA TOWN HALL ABA SOUTH LGA
10,AKWA IBOM,AKWA IBOM NORTH EAST,SD/007/AK,"ETINAN, IBESIKPO ASUTAN, IBIONO IBOM, ITU, NSI...",INEC OFFICE UYO LGA
11,AKWA IBOM,AKWA IBOM NORTH WEST,SD/008/AK,"ABAK, ESSIEN UDIM, ETIM EKPO, IKA, IKONO, IKOT...",INEC OFFICE IKOT EKPENELGA


,state,senatorial_district,code,composition,collation_centre
243,YOBE,YOBE SOUTH,SD/105/YB,"FIKA, FUNE, NANGERE, POTISKUM, DAMATURU",FEDERAL COLLEGE OF EDUCATION
248,ZAMFARA,ZAMFARA NORTH,SD/106/ZF,"K/NAMODA, SHINKAFI, ZURMI, BIRNIN MAGAJI, T/MA...",INEC OFFICE KAURA NAMODA
249,ZAMFARA,ZAMFARA CENTRAL,SD/107/ZF,"GUSAU, TSAFE, BUNGUDU, MARU",INEC OFFICE GUSAU
250,ZAMFARA,ZAMFARA WEST,SD/108/ZF,"MARADUN, BAKURA, ANKA, BUKKUYUM, GUMMI",INEC OFFICE TALATA MAFARA
261,FCT,FEDERAL CAPITAL TERRITORY,SD/109/FCT,"ABAJI AREA COUNCIL, GWAGWA AREA COUNCIL, KUJE ...",INEC OFFICE FCT


Now our dataframe looks tidy!

---
**But looks can be deceiving. I will double-check manually to make sure everything is as it should be. First I'll check how many senatorial districts there are for each state.**

In [59]:
print("Summary of senatorial districts per state:\n")
display(summary)

Summary of senatorial districts per state:



,state,num_districts
0,ANAMBRA,3
1,ABIA,3
2,AKWA IBOM,6
3,BAUCHI,3
4,BAYELSA,6
5,BORNO,3
6,CROSS RIVER,3
7,DELTA,3
8,EBONYI,3
9,EDO,3


We see above that some states have 6 senatorial districts instead of 3. Let's take a microscopic look at the data

In [60]:
# Show all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

display(clean_df)


,state,senatorial_district,code,composition,collation_centre
4,ABIA,ABIA NORTH,SD/001/AB,"UMUNNEOCHI, ISUKWUATO, OHAFIA, AROCHUKWU, BENDE",COUNCIL HALL OHAFIA LGA HQS
5,ABIA,ABIA CENTRAL,SD/002/AB,"UMUAHIA NORTH, UMUAHIA SOUTH, IKWUANO, ISIALA ...",COUNCIL HALL UMUAHIA NORTH LGA HQS
6,ABIA,ABIA SOUTH,SD/003/AB,"ABA NORTH, ABA SOUTH, UGWUNAGBO, OBINGWA, UKW...",ABA TOWN HALL ABA SOUTH LGA
10,AKWA IBOM,AKWA IBOM NORTH EAST,SD/007/AK,"ETINAN, IBESIKPO ASUTAN, IBIONO IBOM, ITU, NSI...",INEC OFFICE UYO LGA
11,AKWA IBOM,AKWA IBOM NORTH WEST,SD/008/AK,"ABAK, ESSIEN UDIM, ETIM EKPO, IKA, IKONO, IKOT...",INEC OFFICE IKOT EKPENELGA
12,AKWA IBOM,AKWA IBOM SOUTH,SD/009/AK,"EASTERN OBOLO, EKET, ESIT EKET, IBENO, IKOT AB...",EKET LGA INEC OFFICE
16,AKWA IBOM,ADAMAWA NORTH,SD/004/AD,"MADAGALI, MAIHA, MICHIKA, MUBI NORTH, MUBI SOUTH",GSS MUBI
17,AKWA IBOM,ADAMAWA SOUTH,SD/005/AD,"DEMSA, GANYE, GUYUK, JADA, MAYO-BELWA, NUMAN, ...",GSS NUMAN
18,AKWA IBOM,ADAMAWA CENTRAL,SD/006/AD,"HONG, FUFORE, SONG, YOLA NORTH, YOLA SOUTH, GERFI",GMMC YOLA
24,ANAMBRA,ANAMBRA NORTH,SD/010/AN,"ONITSHA NORTH, ONITSHA SOUTH, OYI, OGBARU, ANA...",ONITHA NORTH INEC OFFICE


Above, I see that the Adamawa state senatorial districts are stored under "Akwa Ibom". The same is true for a couple of other states.

**That's a flag**

I will correctc this by manually assigning the correct state names to the mismatched senatorial districts.

In [61]:
# Fix mismatched state names manually
fixes = {
    16: "ADAMAWA",
    17: "ADAMAWA",
    18: "ADAMAWA",
    47: "BENUE",
    48: "BENUE",
    49: "BENUE",
    177: "NASARAWA",
    178: "NASARAWA",
    179: "NASARAWA",
}

# Apply corrections
for idx, correct_state in fixes.items():
    clean_df.at[idx, "state"] = correct_state

# Now let's check again
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

display(clean_df)

,state,senatorial_district,code,composition,collation_centre
4,ABIA,ABIA NORTH,SD/001/AB,"UMUNNEOCHI, ISUKWUATO, OHAFIA, AROCHUKWU, BENDE",COUNCIL HALL OHAFIA LGA HQS
5,ABIA,ABIA CENTRAL,SD/002/AB,"UMUAHIA NORTH, UMUAHIA SOUTH, IKWUANO, ISIALA ...",COUNCIL HALL UMUAHIA NORTH LGA HQS
6,ABIA,ABIA SOUTH,SD/003/AB,"ABA NORTH, ABA SOUTH, UGWUNAGBO, OBINGWA, UKW...",ABA TOWN HALL ABA SOUTH LGA
10,AKWA IBOM,AKWA IBOM NORTH EAST,SD/007/AK,"ETINAN, IBESIKPO ASUTAN, IBIONO IBOM, ITU, NSI...",INEC OFFICE UYO LGA
11,AKWA IBOM,AKWA IBOM NORTH WEST,SD/008/AK,"ABAK, ESSIEN UDIM, ETIM EKPO, IKA, IKONO, IKOT...",INEC OFFICE IKOT EKPENELGA
12,AKWA IBOM,AKWA IBOM SOUTH,SD/009/AK,"EASTERN OBOLO, EKET, ESIT EKET, IBENO, IKOT AB...",EKET LGA INEC OFFICE
16,ADAMAWA,ADAMAWA NORTH,SD/004/AD,"MADAGALI, MAIHA, MICHIKA, MUBI NORTH, MUBI SOUTH",GSS MUBI
17,ADAMAWA,ADAMAWA SOUTH,SD/005/AD,"DEMSA, GANYE, GUYUK, JADA, MAYO-BELWA, NUMAN, ...",GSS NUMAN
18,ADAMAWA,ADAMAWA CENTRAL,SD/006/AD,"HONG, FUFORE, SONG, YOLA NORTH, YOLA SOUTH, GERFI",GMMC YOLA
24,ANAMBRA,ANAMBRA NORTH,SD/010/AN,"ONITSHA NORTH, ONITSHA SOUTH, OYI, OGBARU, ANA...",ONITHA NORTH INEC OFFICE


In [62]:
# and finally a double-check of the summary

summary = clean_df.groupby("state")["senatorial_district"].count().reset_index()
summary = summary.rename(columns={"senatorial_district": "num_districts"})
display(summary)

,state,num_districts
0,ANAMBRA,3
1,ABIA,3
2,ADAMAWA,3
3,AKWA IBOM,3
4,BAUCHI,3
5,BAYELSA,3
6,BENUE,3
7,BORNO,3
8,CROSS RIVER,3
9,DELTA,3


That worked. The new cleaned datasets are saved as "cleaned_LGA_SEN.csv" and "cleaned_LGA_SEN.xlsx" under the same folder.

#### Task 2 - 5
I will use Python to automate the data joining, aggregration, and exporting process. First, confirm/install deps:

`pip install geopandas`

Next, I created a `join_data.py` file inside the "Assignment 1" folder. This file contains the script that preprocesses the datasets and consolidates them for use by our QGIS project.

The below command runs the script:

`cd "C:\Users\USER\Desktop\eHealthOrg\Assignment 1"`

`python join_data.py`

#### Task 6 - 7
Delivered as required.